- Ex 1 : nous devons résoudre le prolème en faisant la comparaison entre A* et un algorithem de notre choix. Nous avons pris bfs car c'est une algorithem simple mais  quand-même optimal.

In [2]:
import heapq
import time
import math

graph_ex = {
    (0, 0): [((0, 1), 1), ((1, 0), 1)],
    (0, 1): [((0, 0), 1), ((0, 3), 1), ((1, 1), 1)],
    (0, 3): [((0, 1), 1), ((0, 4), math.inf), ((1, 3), 1)],
    (0, 4): [((0, 3), math.inf), ((1, 4), 1)],

    (1, 0): [((0, 0), 1), ((1, 1), 1), ((3, 0), 1)],
    (1, 1): [((1, 0), 1), ((0, 1), 1), ((1, 3), 1), ((3, 1), 1)],
    (1, 3): [((1, 1), 1), ((0, 3), 1), ((1, 4), 1), ((3, 3), 1)],
    (1, 4): [((1, 3), 1), ((0, 4), 1), ((3, 4), 1)],

    (3, 0): [((1, 0), 1), ((3, 1), 1), ((4, 0), 1)],
    (3, 1): [((3, 0), 1), ((1, 1), 1), ((3, 3), 1), ((4, 1), 1)],
    (3, 2): [((3, 3), math.inf)],
    (3, 3): [((3, 1), 1), ((1, 3), 1), ((3, 4), 1), ((4, 3), 1), ((3, 2), math.inf)],
    (3, 4): [((3, 3), 1), ((1, 4), 1), ((4, 4), 1)],

    (4, 0): [((3, 0), 1), ((4, 1), 1)],
    (4, 1): [((4, 0), 1), ((3, 1), 1), ((4, 3), 1)],
    (4, 3): [((4, 1), 1), ((3, 3), 1), ((4, 4), 1)],
    (4, 4): [((4, 3), 1), ((3, 4), 1)]
}

# Breadth-First Search (BFS)
def bfs(graph, start, goal):
    queue = [(start, [start])]
    visited = set()
    visited.add(start)
    nodes_explored = 0

    while queue:
        current, path = queue.pop(0)
        nodes_explored += 1

        if current == goal:
            return path, nodes_explored

        for neighbor, cost in graph.get(current, []):
            if neighbor not in visited and cost != math.inf:
                visited.add(neighbor)
                queue.append((neighbor, path + [neighbor]))

    return None, nodes_explored  # No path found

# nous prenons la distance de Manhattan comme heuristique
def heuristic(node, goal):
    return abs(goal[0] - node[0]) + abs(goal[1] - node[1])


# A* Search
def astar(graph, start, goal):
    open_set = []
    heapq.heappush(open_set, (0, start, [start]))
    g_scores = {start: 0}
    nodes_explored = 0

    while open_set:
        _, current, path = heapq.heappop(open_set)
        nodes_explored += 1

        if current == goal:
            return path, nodes_explored

        for neighbor, cost in graph.get(current, []):
            if cost == math.inf:
                continue

            tentative_g_score = g_scores[current] + cost
            if neighbor not in g_scores or tentative_g_score < g_scores[neighbor]:
                g_scores[neighbor] = tentative_g_score
                f_score = tentative_g_score + heuristic(neighbor, goal)
                heapq.heappush(open_set, (f_score, neighbor, path + [neighbor]))

    return None, nodes_explored  # No path found


#nous comparosn 
start, goal = (0, 0), (4, 4)

# BFS Execution
start_time = time.time()
bfs_path, bfs_nodes = bfs(graph_ex, start, goal)
bfs_time = time.time() - start_time

# A* Execution
start_time = time.time()
astar_path, astar_nodes = astar(graph_ex, start, goal)
astar_time = time.time() - start_time

# Results
print("Chemin bfs:", bfs_path)
print("BFS Nodes Explored:", bfs_nodes)
print("BFS Execution Time:", bfs_time)

print("\n")

print("\nA* Path:", astar_path)
print("A* Nodes Explored:", astar_nodes)
print("A* Execution Time:", astar_time)

BFS Path: [(0, 0), (0, 1), (0, 3), (1, 3), (1, 4), (3, 4), (4, 4)]
BFS Nodes Explored: 16
BFS Execution Time: 4.410743713378906e-05

A* Path: [(0, 0), (0, 1), (0, 3), (1, 3), (3, 3), (3, 4), (4, 4)]
A* Nodes Explored: 8
A* Execution Time: 4.291534423828125e-05


- Comme discuté en TP02, bfs est optiaml. Nous voyons que A* est plus efficace ( moins de noeuds visité et un peu plus rapide) Cela est du au fait que l'heuriste nous permet de supprimer la visite de certain noeuds 

- Ex 2 : Pour notre projet, nous devons appliquer les algorithme étudiés, les algorithme de recherche,  à un problème. 

Définission du problème : Vous êtes un étudiants, en cours d'informatique, et vous devez solliciter l'expertise d'un assistant. Vous avez  une pose de 1 quart d'heure à disposition et vous  devez trouver le chemin le plus cours pour avoir le plus de temps possible avec l'assistant. L'université est vu comme un emsemble d'intersection de couloir ( des noeuds) que connecter par des couloir (ponderé par leur disntance ). Le coût associer à chaque traverser de couloir correspond aux temps qu'il a fallu utiliser pour le traverser. Cepdant, il se trouver répartit, dans l'université, des machine à café qui, en consommant un café, nous rendent efficace comme si nous gagnions 10 minutes. Cependant, nous ne pouvons pas utuilsier chaque machine à café plus d'une fois sans faire d'overdose de café

(nous pourions par la suite integré des variations au problème , par exemble avec des évéenment probalbilte apparaissant dans l'université , pour par exemple quand nous devrions continuer le projet avec des chaâines de markov)

In [2]:
import heapq
import time
import math

# Graphe de l'université avec intersections, couloirs pondérés et machines à café
university_graph = {
    (0, 0): [((0, 1), 2), ((1, 0), 3)],
    (0, 1): [((0, 0), 2), ((0, 2), 3), ((1, 1), 2)],
    (0, 2): [((0, 1), 3), ((0, 3), 1), ((1, 2), 2)],
    (0, 3): [((0, 2), 1), ((0, 4), 2), ((1, 3), 3)],  # Machine à café
    (0, 4): [((0, 3), 2), ((1, 4), 3)],
    (1, 0): [((0, 0), 3), ((1, 1), 2), ((2, 0), 3)],  
    (1, 1): [((1, 0), 2), ((0, 1), 2), ((1, 2), 1), ((2, 1), 2)],
    (1, 2): [((1, 1), 1), ((0, 2), 2), ((1, 3), 2), ((2, 2), 2)],
    (1, 3): [((1, 2), 2), ((0, 3), 3), ((1, 4), 1), ((2, 3), 2)],
    (1, 4): [((1, 3), 1), ((0, 4), 3), ((2, 4), 2)],  # Machine à café
    (2, 0): [((1, 0), 3), ((2, 1), 1)],
    (2, 1): [((2, 0), 1), ((1, 1), 2), ((2, 2), 2), ((3, 1), 2)],
    (2, 2): [((2, 1), 2), ((1, 2), 2), ((2, 3), 1), ((3, 2), 3)],
    (2, 3): [((2, 2), 1), ((1, 3), 2), ((2, 4), 3), ((3, 3), 2)],
    (2, 4): [((2, 3), 3), ((1, 4), 2), ((3, 4), 2)],
    (3, 1): [((2, 1), 2), ((3, 2), 2)],
    (3, 2): [((3, 1), 2), ((2, 2), 3), ((3, 3), 1)],
    (3, 3): [((3, 2), 1), ((2, 3), 2), ((3, 4), 2)],
    (3, 4): [((3, 3), 2), ((2, 4), 2)]  # Bureau de l'assistant
}

# Machines à café
coffee_machines = {(0, 3), (1, 4)}

# Position de départ et d'arrivée
start = (0, 0)
goal = (3, 4)

# Heuristique (Manhattan)
def heuristic(node, goal):
    return abs(goal[0] - node[0]) + abs(goal[1] - node[1])

# A* modifié pour gérer les machines à café
def astar(graph, start, goal, coffee_machines):
    open_set = []
    heapq.heappush(open_set, (0, start, frozenset(), [start]))  # (score, position, cafés visités, chemin)
    g_scores = {(start, frozenset()): 0}  # Score de coût minimum atteint avec cet état
    nodes_explored = 0

    while open_set:
        _, current, visited_coffees, path = heapq.heappop(open_set)
        nodes_explored += 1

        if current == goal:
            return path, nodes_explored

        for neighbor, cost in graph.get(current, []):
            new_visited = visited_coffees
            new_cost = g_scores[(current, visited_coffees)] + cost
            
            # Si c'est une machine à café non encore visitée, appliquer le bonus (-10)
            if neighbor in coffee_machines and neighbor not in visited_coffees:
                new_cost -= 10
                new_visited = visited_coffees | {neighbor}  # Ajouter la machine visitée

            state = (neighbor, new_visited)
            if state not in g_scores or new_cost < g_scores[state]:
                g_scores[state] = new_cost
                f_score = new_cost + heuristic(neighbor, goal)
                heapq.heappush(open_set, (f_score, neighbor, new_visited, path + [neighbor]))

    return None, nodes_explored  # Aucun chemin trouvé

# Exécution
start_time = time.time()
astar_path, astar_nodes = astar(university_graph, start, goal, coffee_machines)
astar_time = time.time() - start_time

# Résultats
print("A* Path:", astar_path)
print("A* Nodes Explored:", astar_nodes)
print("A* Execution Time:", astar_time)

A* Path: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4)]
A* Nodes Explored: 12
A* Execution Time: 0.00013113021850585938


voilà notre petit code